In [90]:
import pandas as pd
import numpy as np

In [91]:
encoded = pd.read_csv('../Datasets/ecoded.csv',sep=';')
encoded

,embedding,gt,product_id
0,[-0.07581986 -1.0008311 -0.17027485 -0.30064806],35.0,6909
1,[-0.12546918 -0.99800307 -0.12940243 -0.32935148],70.0,6909
2,[-0.11825792 -0.99960494 -0.13386402 -0.32566485],137.0,6909
3,[-0.15743713 -0.9984799 -0.10203394 -0.3480346 ],274.0,6909
4,[-0.14252292 -0.9991128 -0.11403638 -0.3395455 ],333.0,6909
...,...,...,...
7704,[-0.0235827 -1.0026313 -0.2061916 -0.27344364],44.0,247737
7705,[-0.08334108 -1.0014222 -0.15993714 -0.306581 ],30.0,247737
7706,[-0.07737647 -1.0026553 -0.16350313 -0.30359122],37.0,247737
7707,[-0.11576617 -1.0017638 -0.13224027 -0.32551962],65.0,247737


In [92]:
encoded.dtypes

embedding      object
gt            float64
product_id      int64
dtype: object

In [93]:
encoded.embedding = encoded.embedding.map(lambda x: [float(num) for num in x[1:-1].split(' ') if num != ''])
encoded.embedding

0       [-0.07581986, -1.0008311, -0.17027485, -0.3006...
1       [-0.12546918, -0.99800307, -0.12940243, -0.329...
2       [-0.11825792, -0.99960494, -0.13386402, -0.325...
3       [-0.15743713, -0.9984799, -0.10203394, -0.3480...
4       [-0.14252292, -0.9991128, -0.11403638, -0.3395...
                              ...                        
7704    [-0.0235827, -1.0026313, -0.2061916, -0.27344364]
7705    [-0.08334108, -1.0014222, -0.15993714, -0.306581]
7706    [-0.07737647, -1.0026553, -0.16350313, -0.3035...
7707    [-0.11576617, -1.0017638, -0.13224027, -0.3255...
7708    [-0.09968835, -1.0027409, -0.14507854, -0.3163...
Name: embedding, Length: 7709, dtype: object

In [94]:
embeddings = pd.DataFrame(encoded.embedding.to_list(), columns=['embedding_1','embedding_2','embedding_3','embedding_4'])
display(embeddings)
encoded = pd.concat([encoded,embeddings],axis=1).drop(columns=['embedding'])
encoded

,embedding_1,embedding_2,embedding_3,embedding_4
0,-0.075820,-1.000831,-0.170275,-0.300648
1,-0.125469,-0.998003,-0.129402,-0.329351
2,-0.118258,-0.999605,-0.133864,-0.325665
3,-0.157437,-0.998480,-0.102034,-0.348035
4,-0.142523,-0.999113,-0.114036,-0.339546
...,...,...,...,...
7704,-0.023583,-1.002631,-0.206192,-0.273444
7705,-0.083341,-1.001422,-0.159937,-0.306581
7706,-0.077376,-1.002655,-0.163503,-0.303591
7707,-0.115766,-1.001764,-0.132240,-0.325520


,gt,product_id,embedding_1,embedding_2,embedding_3,embedding_4
0,35.0,6909,-0.075820,-1.000831,-0.170275,-0.300648
1,70.0,6909,-0.125469,-0.998003,-0.129402,-0.329351
2,137.0,6909,-0.118258,-0.999605,-0.133864,-0.325665
3,274.0,6909,-0.157437,-0.998480,-0.102034,-0.348035
4,333.0,6909,-0.142523,-0.999113,-0.114036,-0.339546
...,...,...,...,...,...,...
7704,44.0,247737,-0.023583,-1.002631,-0.206192,-0.273444
7705,30.0,247737,-0.083341,-1.001422,-0.159937,-0.306581
7706,37.0,247737,-0.077376,-1.002655,-0.163503,-0.303591
7707,65.0,247737,-0.115766,-1.001764,-0.132240,-0.325520


In [95]:
X = encoded.drop(columns=['gt','product_id'])
Y = encoded['gt']
X
Y

0        35.0
1        70.0
2       137.0
3       274.0
4       333.0
        ...  
7704     44.0
7705     30.0
7706     37.0
7707     65.0
7708     79.0
Name: gt, Length: 7709, dtype: float64

In [96]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test   = train_test_split(X,Y,shuffle=True,random_state=69)

In [97]:
import pickle
with open('best_params_xgboost_2.pkl','rb') as file2, open('best_params_xgboost.pkl','rb') as file1:
    best_params1 = pickle.load(file1)
    best_params2 = pickle.load(file2)
best_params1


{'colsample_bytree': 0.4,
 'gamma': 0.0,
 'learning_rate': 0.1,
 'max_depth': 10,
 'min_child_weight': 2,
 'n_estimators': 400}

In [98]:
from sklearn.ensemble import GradientBoostingRegressor

xgboost_encoded = GradientBoostingRegressor(**best_params2)

result_f_sklearn_gooddf = xgboost_encoded.fit(X_train, y_train)

In [99]:
from sklearn.metrics import mean_squared_error

y_predicted_test = xgboost_encoded.predict(X_test)
y_predicted_train = xgboost_encoded.predict(X_train)

rms_test = mean_squared_error(y_test, y_predicted_test, squared=False)
rms_train = mean_squared_error(y_train, y_predicted_train, squared=False)
print(f'train: {rms_train}\ntest: {rms_test}')

train: 66.68456764837829
test: 111.94892310939883


In [100]:
from sklearn.model_selection import GridSearchCV

gsc1 = GridSearchCV(
            estimator=GradientBoostingRegressor(),
            param_grid={"n_estimators": list(range(60,80,10)),
                        "learning_rate": [0.1,0.05],
                        "max_depth": [5,10],#,20],
                        'min_impurity_decrease':[2],#,10],
                        'min_samples_split':[50,100],
                        'min_samples_leaf':[10,20],
                        'max_features':['auto'],
                        'max_leaf_nodes':list(range(100,300,100)),
                        'ccp_alpha':[0.1,0.5]},
            cv=5, scoring='neg_root_mean_squared_error', verbose=10, n_jobs=-1)

grid_result_encoded = gsc1.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 128 candidates, totalling 640 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:   17.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 197 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done 218 tasks      | elapsed:   21.2s
[Paralle

In [101]:
print(f'train:{grid_result_encoded.score(X_train,y_train)}\ntest:{grid_result_encoded.score(X_test, y_test)}')
results = (grid_result_encoded.score(X_train,y_train), grid_result_encoded.score(X_test, y_test))
# rmse.append(results)

train:-91.48912846917312
test:-106.76120430417743


In [102]:
grid_result_encoded.best_params_

{'ccp_alpha': 0.5,
 'learning_rate': 0.05,
 'max_depth': 10,
 'max_features': 'auto',
 'max_leaf_nodes': 100,
 'min_impurity_decrease': 2,
 'min_samples_leaf': 20,
 'min_samples_split': 50,
 'n_estimators': 60}